<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/neural_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Language Modeling

In [6]:
from utils import ngram_utils as utils
from utils.amazon_dataset import AmazonDataset
from utils.neural_lm import BagOfNGrams, DecoderMLP, seq2seq
import torch

In [7]:
torch.manual_seed(1)


In [8]:
# Read data from .txt files and create lists of reviews

train_data = []
# create a list of all the reviews 
with open('../data/amazon_reviews_clothing_train.txt', 'r') as f:
    train_data = [review for review in f.read().split('\n') if review]

test_data = []
# create a list of all the reviews 
with open('../data/amazon_reviews_clothing_test.txt', 'r') as f:
    test_data = [review for review in f.read().split('\n') if review]
    
valid_data = []
# create a list of all the reviews 
with open('../data/amazon_reviews_clothing_valid.txt', 'r') as f:
    valid_data = [review for review in f.read().split('\n') if review]
    

In [ ]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = utils.tokenize_dataset(train_data)
test_data_tokenized, all_tokens_test = utils.tokenize_dataset(test_data)
valid_data_tokenized, all_tokens_valid = utils.tokenize_dataset(valid_data)


In [2]:
# n = 1
train_padded_uni = utils.pad_sentences(train_data_tokenized, 1)
train_unigram = utils.find_ngrams(train_padded_uni, 1)
train_vocab_unigram, train_count_unigram = utils.ngram_counts(train_unigram)
train_id2token_unigram, train_token2id_unigram = utils.ngram_dict(train_vocab_unigram)

valid_padded_uni = utils.pad_sentences(valid_data_tokenized, 1)
valid_unigram = utils.find_ngrams(valid_padded_uni, 1)
valid_vocab_unigram, count_unigram = utils.ngram_counts(valid_unigram)
valid_id2token_unigram, valid_token2id_unigram = utils.ngram_dict(valid_vocab_unigram)

NameError: name 'utils' is not defined

In [ ]:
N = 10
train_data_id = utils.create_data_id(train_unigram, train_token2id_unigram)
train_data_id_merged = utils.create_data_id_merged(train_data_id, train_token2id_unigram, N)

valid_data_id = utils.create_data_id(valid_unigram, valid_token2id_unigram)
valid_data_id_merged = utils.create_data_id_merged(valid_data_id, valid_token2id_unigram, N)

In [ ]:
train_dataset = AmazonDataset(train_data_id_merged, max_inp_length=None, use_cuda=True)
train_loader = DataLoader(train_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

valid_dataset = AmazonDataset(valid_data_id_merged, max_inp_length=None, use_cuda=True)
valid_loader = DataLoader(valid_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

In [ ]:
train_data_id_merged

In [ ]:
train_dataset[0], train_dataset[0][0].shape, \
valid_dataset[0], valid_dataset[0][0].shape

In [ ]:
num_epochs = 500
encoder = BagOfNGrams(len(train_id2token_unigram), emb_dim=300, hidden_size=256, out_size=128, activation='Tanh', nlayers=1, reduce='mean', dropout=0.0, batch_norm=False)
decoder = DecoderMLP(input_size=128, output_size=len(train_id2token_unigram), hidden_size=256)
model = seq2seq(encoder, decoder, use_cuda=False, lr=1e-1)

for epoch in range(num_epochs):
    # Train
    train_loss_epoch = 0
    for i, (data, labels) in _tqdm(enumerate(train_loader)):
        prediction, loss = model.train_step(data, labels)
        train_loss_epoch += loss
    if epoch % 10 == 0:
        print("Epoch {}: Loss {}".format(epoch, train_loss_epoch))
#         print("prediction ", prediction)

        generated = model.eval_step()
        generated_str = ' '.join([g[0] for g in generated])
        print("Generated Sentence: ", generated_str)
        
